# Régularisation des pronoms

In [1]:
def d_in_pos(csv_lu):
    
    """
    Fonction récupérant le contenu d'un CSV avec les colones suivantes :
        - "Numero de token"
        - "Mot forme"
        - "Validation automatique"
        - "Modifié"
        - "Lemme(s)"
        - "POS"
        
    Cette fonction boucle sur chaque token et chaque fois garde un trio de tokens
    dont les numéros se suivent (en excluant la ponctuation). Puis, sur ce trio, on
    teste des conditions, dont la principale est qu'Analog a hésité entre une forme
    de déterminant et une forme de pronom, mais ni préposition, ni S, et que
    le mot suivant est une forme de ponctuation ou un verbe.
    
    La fonction ajoute alors la nouvelle forle dictionnaire du token comme valeur.
    Une fois tous les tokens analysés, la fonction retourne le dictionnaire contenant
    tous les tokens modifiés.
    
    :param csv_lu: Le contenu du fichier CSV sur lequel on souhaite
            effectuer des vérifications, extrait et lu par la fonction principale.
            
    :return: Retourne un dictionnaire dont les clés sont les numéros des tokens
            contenus et les valeurs la ligne entière du CSV modifiée par la fonction.
    
    """
    
    
    # Création de dictionnaires locaux, vides.
    dico_mots = {}
    dico_modifies = {}
    # Compteur de tokens modifiés, compteur du total de tokens.
    total = 0
    compteur = 0
    # Liste des valeurs PoS à rechercher dans le mot suivant.
    suivant_pos = ['Vuc', 'Vun', 'Vvc', 'Vvn', 'Fw', 'Fs']
    # Liste des valeurs PoS correspondant à des déterminants.
    D_pos = ['Da', 'Dd', 'Ds', 'Dn', 'Dp', 'Di', 'Dr', 'Dt']
    P_pos = ['Pp', 'Pd', 'Pi', 'Ps', 'Pt', 'Pr']
    # Liste destinée à rassembler les valeurs PoS à enlever d'une liste de PoS
    # le cas échéant.
    a_enlever = []
    separateur = "|"
    
    # On boucle sur les lignes du CSV d'entrée pour créer
    # un dictionnaire général (dico_mots) à exploiter.
    for row in csv_lu:
            
        # On transfère les clés et valeurs dans un dictionnaire local,
        # de manière à avoir un dictionnaire général avec
        # clé = numéro d'ordre des tokens (pas identifiant des tokens)
        # valeurs = dictionnaire identique à la ligne du CSV d'origine.
        
        identifiant = int(row['Numero de token'])
        
                # On ajoute l'unité au dictionnaire général,
                # puis on remet le dictionnaire local à vide,
                # (j'ai eu de grosses erreurs avant d'y penser).
        dico_mots[identifiant] = row
        total += 1
        
                
    for identifiant in dico_mots.keys():
        if identifiant + 1 in dico_mots.keys():
            
            # On crée deux variables sur le mot-cible et celui
            # qui le suit.
            # À la fin des tests, le mot_cible uniquement sera
            # susceptible d'être modifié.
            
            mot_cible = dico_mots[identifiant]
            mot_suivant = dico_mots[identifiant + 1]
            
            # Si Analog a hésité entre une forme de déterminant
            # et une forme de pronom, mais ni préposition, ni S,
            # et que le mot suivant est une forme de ponctuation ou un verbe, on continue.
            if "D" in mot_cible['POS'] and "P" in mot_cible['POS'] and "PREP" not in mot_cible['POS'] and "S" not in mot_cible['POS'] and mot_suivant["POS"] in suivant_pos:
        
                # On crée des listes pour les PoS et les lemmes, pour pouvoir comparer les
                # différentes valeurs séparément aux listes déclarées au début de la fonction.
                liste_pos = mot_cible['POS'].split("|")
                liste_lemmes = mot_cible["Lemme(s)"].split("|")
                
                # On compare chaque valeur avec la liste des PoS à garder
                # et on fait une nouvelle liste avec ceux qui ne correspondent pas.
                for valeur in liste_pos:
                    if valeur not in P_pos:
                        a_enlever.append(valeur)
                        
                # On enlève de la liste de PoS possibles du mot-cible toute
                # valeur correspondant aux PoS pertinents et les lemmes qui vont avec.
                for valeur in a_enlever:
                    if "|" in mot_cible['Lemme(s)']:
                        index_pos = liste_pos.index(valeur)
                        lemme_en_moins = liste_lemmes[index_pos]
                        liste_lemmes.remove(lemme_en_moins)
                        
                    liste_pos.remove(valeur)
                    
            
                mot_cible['Lemme(s)'] = separateur.join(liste_lemmes)
                mot_cible['POS'] = separateur.join(liste_pos)
                a_enlever = []
                
                # Le token a été modifié, donc on ajoute 1 au compteur de mots modifiés.
                compteur += 1
                
                # On ajoute le token modifié au dictionnaire retourné par la fonction.
                dico_modifies[mot_cible['Numero de token']] = mot_cible
                
                
    # On calcule les totaux et pourcentages de tokens modifiés par la fonction.
    pourcent_traites = compteur * 100 / total
    print(compteur, ":", round(pourcent_traites,1), "%")
    
    # On retourne les tokens modifiés.
    return dico_modifies

In [2]:
def desambiguiser(csv_entree, csv_sortie):
    
    """
        Fonction récupérant un CSV dont chaque ligne est un token lemmatisé,
        avec les colonnes telles quelles :
        
            - "Numero de token" --- étant le numéro constituant la valeur du @n
                    de l'élément <w> dans le XML dont les tokens ont été extraits
                    (en principe, chaque token a un numéro unique).
                    
            - "Mot forme" --- étant le texte de l'élément <w> original, avec
                    abréviations résolues et graphie modernisée s'il y a lieu.
                    
            - "Validation automatique" --- contient soit "yes", soit "no" ; la
                    valeur est produite par Analog lors de la lemmatisation et
                    exprime si Analog est certain d'une interprétation particulière.
            
            - "Modifié" --- contient soit "yes", soit "no" ; la valeur a été produite
                    par les scripts précédents (NV n°4-6) et
                    indique si les valeurs de la ligne sont exactement celles produites
                    par Analog ("no") ou si le script précédent les a modifiées ("yes").
                    
            - "Lemme(s)" --- étant le lemme préconisé par Analog, ou bien les lemmes
                    entre lesquels Analog hésite, séparés par des '|'.
                    
            - "POS" --- étant la valeur POS préconisée par Analog, ou bien les
                    différentes valeurs entre lesquelles Analog hésite, séparées par
                    des '|'.
        
        La fonction crée à partir du CSV d'entrée un dictionnaire qu'elle transmet à la
        fonction secondaire chargée d'analyser les tokens et les modifier si besoin et
        récupère un dictionnaire contenant uniquement les tokens modifiés.
        
        Puis elle produit un autre fichier CSV. Si le token se trouve dans le dictionnaire
        retourné par la fonction secondaire, c'est cette version qui sera écrite dans le
        nouveau CSV. Sinon, la version originale sera copiée.
        
        :param csv_entree: Le chemin interne du fichier CSV sur lequel on souhaite
            effectuer des vérifications.
            
        :param csv_sortie: Le chemin interne du fichier CSV dans lequel écrire
            les tokens modifiés.
            
    """
        
    import csv
    colonnes = [
            'Numero de token',
            'Mot forme',
            'Validation automatique',
            'Modifié',
            'Lemme(s)',
            'POS'
        ]
    row_modifie = {}
    
    # Ouvrir le CSV d'Analog et le lire.
    with open(csv_entree) as csv_a_lire:
        csv_lu = csv.DictReader(csv_a_lire)
        
        # Récupérer la liste des tokens contenant D dans le POS
        # et modifiés par la fonction.
        # La fonction retourne un dictionnaire ne contenant que
        # les nouvelles formes des tokens modifiés.
        dico_modifies = d_in_pos(csv_lu)
        
        
    # Rouvrir le CSV d'entrée et le lire.
    with open(csv_entree) as csv_a_lire:
        csv_lu = csv.DictReader(csv_a_lire)

        # Ouvrir le fichier de sortie et le lire en mode écriture.
        with open(csv_sortie, 'w') as csv_a_ecrire:
            a_ecrire = csv.DictWriter(csv_a_ecrire, fieldnames = colonnes)
            # Écrire les noms des colonnes.
            a_ecrire.writeheader()
            
            # Boucler à nouveau sur les lignes du CSV d'entrée.
            for row in csv_lu:
                
                # Si le token se trouve dans les formes modifiées,
                # on écrit la nouvelle forme dans le nouveau CSV.
                if row['Numero de token'] in dico_modifies.keys():
                    row_modifie = dico_modifies[row['Numero de token']]
                    
                    a_ecrire.writerow({
                        'Numero de token' : row_modifie['Numero de token'],
                        'Mot forme' : row_modifie['Mot forme'],
                        'Validation automatique' : row_modifie['Validation automatique'],
                        'Modifié' : "yes",
                        'Lemme(s)' : row_modifie['Lemme(s)'],
                        'POS' : row_modifie['POS']
                    })
                    row_modifie = {}
                
                # Si le token n'était pas dans les formes modifiées,
                # on écrit la forme originelle.
                else:
                    a_ecrire.writerow({
                        'Numero de token' : row['Numero de token'],
                        'Mot forme' : row['Mot forme'],
                        'Validation automatique' : row['Validation automatique'],
                        'Modifié' : row['Modifié'],
                        'Lemme(s)' : row['Lemme(s)'],
                        'POS' : row['POS']
                    })

In [3]:
desambiguiser('/home/erminea/Documents/CONDE/GC/GC-desambig-3-det.csv',
             '/home/erminea/Documents/CONDE/GC/GC-desambig-4-pro.csv')

374 : 0.6 %
